In [1]:
import pandas as pd
import numpy as np

import os
import glob

In [2]:
from transformers import pipeline, AutoTokenizer
from google.colab import drive

In [3]:
!pip install -q wrds psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00


In [4]:
import datetime as dt

In [5]:
from datasets import load_dataset
import random
from datetime import datetime, timedelta

import wrds
import re

# 1. Acquire FINRA Short Interest Data

In [6]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Step 2: Set your actual Google Drive path (update as needed)
folder_path = '/content/drive/MyDrive/Projects/JPM Short Interest/Data/Short Interest FINRA'

# Step 3: Read all matching CSV files
csv_files = glob.glob(os.path.join(folder_path, 'shrt*.csv'))

# Step 4: Load and concatenate
finra_df_list = []
for file in csv_files:
    finra_df = pd.read_csv(file, sep='|', dtype=str, engine='python', on_bad_lines='skip')
    finra_df_list.append(finra_df)

finra_df = pd.concat(finra_df_list, ignore_index=True)

In [8]:
# Data Cleaning
finra_df_cleaned = finra_df[finra_df['marketClassCode'] != '']

# Show preview
finra_df_cleaned.shape

(2197833, 14)

In [9]:

AAPL = finra_df_cleaned[finra_df_cleaned["symbolCode"] == "AAPL"]
AAPL.head()

,accountingYearMonthNumber,symbolCode,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate
48,20250731,AAPL,Apple Inc. Common Stock,R,NNM,104634941,93946599,NaN,48911215,2.14,NaN,11.38,10688342,2025-07-31
20642,20250715,AAPL,Apple Inc. Common Stock,R,NNM,93946599,110101153,NaN,48526568,1.94,NaN,-14.67,-16154554,2025-07-15
41249,20240731,AAPL,Apple Inc. Common Stock,R,NNM,117696224,135383184,NaN,48246946,2.44,NaN,-13.06,-17686960,2024-07-31
60777,20240715,AAPL,Apple Inc. Common Stock,R,NNM,135383184,132235437,NaN,55946376,2.42,NaN,2.38,3147747,2024-07-15
80351,20230731,AAPL,Apple Inc. Common Stock,R,NNM,105460144,126626604,NaN,52323234,2.02,NaN,-16.72,-21166460,2023-07-31


In [10]:
AAPL.shape

(110, 14)

# 2. Acquire Transcript Data from WRDS

In [9]:
# 1. Connect and discover transcript tables

db = wrds.Connection()  # opens browser auth the first time

# See which library holds transcripts on your site (often 'ciq')
libs = db.list_libraries()
print(libs)

# List candidate tables and pick ones with "transcript" in the name
ciq_tables = db.list_tables(library='ciq')  # try 'ciq' first; if not, print(libs) and try the vendor lib that exists for you
[t for t in ciq_tables if re.search('transcript', t, re.I)]

Enter your WRDS username [root]:beihann2
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common', 'audit_corp_legal', 'auditsmp', 'auditsmp_all', 'bank', 'bank_all', 'bank_premium_samp', 'banksamp', 'block', 'block_all', 'boardex', 'boardex_na', 'boardex_trial', 'boardsmp', 'bvd_amadeus_trial', 'bvd_bvdbankf_trial', 'bvd_orbis_trial', 'bvdsamp', 'calcbench_trial', 'calcbnch', 'candid_samp', 'cboe', 'cboe_all', 'cboe_sample', 'cboesamp', 'cddsamp', 'ciq', 'ciq_common', 'ciq_transcripts', 'ciqsamp', 'ciqsamp_capstrct', 'ciqsamp_common', 'ciqsamp_keydev', 'ciqsamp_pplintel', 'ciqsamp_ratings', 'ciqsamp_transactions', 'ciqsamp_transcripts', 'cisdmsmp', 'columnar', 'comp', 'comp_bank_daily', 'comp_execucomp', 'comp_global_d

['ciqtranscript',
 'ciqtranscriptcollectiontype',
 'ciqtranscriptcomponent',
 'ciqtranscriptcomponenttype',
 'ciqtranscriptdelayreason',
 'ciqtranscriptdelayreasontype',
 'ciqtranscriptperson',
 'ciqtranscriptpresentationtype',
 'ciqtranscriptspeakertype',
 'wrds_transcript_detail',
 'wrds_transcript_person']

In [10]:
db.describe_table(library='ciq_transcripts', table='wrds_transcript_detail')
db.describe_table(library='ciq_transcripts', table='wrds_transcript_person')

Approximately 1747814 rows in ciq_transcripts.wrds_transcript_detail.
Approximately 88087807 rows in ciq_transcripts.wrds_transcript_person.


,name,nullable,type,comment
0,transcriptid,True,"NUMERIC(11, 0)",Transcript ID
1,transcriptcomponentid,True,"NUMERIC(11, 0)",Transcript Component ID
2,componentorder,True,INTEGER,Transcript Component Order
3,transcriptcomponenttypeid,True,INTEGER,Transcript Component Type ID
4,transcriptcomponenttypename,True,VARCHAR(400),Transcript Component Type Name
5,transcriptpersonid,True,"NUMERIC(11, 0)",Transcript Person ID
6,transcriptpersonname,True,VARCHAR(800),Name of Person
7,proid,True,"NUMERIC(11, 0)",Professional ID
8,companyofperson,True,VARCHAR(800),Company of Person
9,speakertypeid,True,INTEGER,Speaker Type ID


In [11]:
# STEP 1: inspect columns + sample rows
cols = db.describe_table('ciq_transcripts', 'ciqtranscript')
print(cols)

sample = db.raw_sql("SELECT * FROM ciq_transcripts.ciqtranscript LIMIT 5;")
print(sample.columns.tolist())
sample.head()

Approximately 1674992 rows in ciq_transcripts.ciqtranscript.
                           name  nullable            type  \
0                      keydevid      True  NUMERIC(11, 0)   
1                  transcriptid      True  NUMERIC(11, 0)   
2    transcriptcollectiontypeid      True         INTEGER   
3  transcriptpresentationtypeid      True         INTEGER   
4     transcriptcreationdateutc      True       TIMESTAMP   
5                audiolengthsec      True  NUMERIC(11, 0)   

                        comment  
0                      keydevid  
1                  transcriptid  
2    transcriptcollectiontypeid  
3  transcriptpresentationtypeid  
4     transcriptcreationdateutc  
5                audiolengthsec  
['keydevid', 'transcriptid', 'transcriptcollectiontypeid', 'transcriptpresentationtypeid', 'transcriptcreationdateutc', 'audiolengthsec']


,keydevid,transcriptid,transcriptcollectiontypeid,transcriptpresentationtypeid,transcriptcreationdateutc,audiolengthsec
0,0.0,2745244.0,13,5,2023-02-20 16:25:24,<NA>
1,7.0,864627.0,6,5,2015-09-08 08:56:36,<NA>
2,202048.0,16557.0,6,5,2009-02-11 03:03:26,<NA>
3,1018019.0,708.0,6,5,2008-06-20 00:11:20,<NA>
4,1026789.0,4672.0,6,5,2008-06-20 00:51:51,<NA>


In [12]:
params = {
    "start": "2023-08-01",
    "end":   "2025-08-01",
}

sql_multi = """
WITH patterns(ticker, kw) AS (
  VALUES

    -- Tesla
    ('TSLA',  '%%Tesla%%'),
    ('TSLA',  '%%TSLA%%'),

  -- Alphabet / Google
    ('GOOGL', '%%Alphabet%%'),
    ('GOOGL', '%%Google%%'),
    ('GOOGL', '%%GOOGL%%')

),
calls AS (
  SELECT
      t.transcriptid,
      t.transcriptcreationdateutc AS event_dt
  FROM ciq_transcripts.ciqtranscript t
  WHERE t.transcriptcreationdateutc BETWEEN %(start)s AND %(end)s
),
joined AS (
  SELECT
      c.transcriptid,
      string_agg(c.componenttext, E'\\n' ORDER BY c.componentorder) AS full_text
  FROM ciq_transcripts.ciqtranscriptcomponent c
  WHERE c.transcriptid IN (SELECT transcriptid FROM calls)
  GROUP BY c.transcriptid
),
hits AS (
  SELECT
      j.transcriptid,
      p.ticker,
      j.full_text
  FROM joined j
  JOIN patterns p
    ON j.full_text ILIKE p.kw
),
dedup AS (
  SELECT DISTINCT ON (h.transcriptid)
      h.transcriptid,
      h.ticker,
      j.full_text
  FROM hits h
  JOIN joined j USING (transcriptid)
  ORDER BY h.transcriptid, h.ticker
),
bucketed AS (
  SELECT
    c.transcriptid,
    c.event_dt,
    d.ticker,
    d.full_text,
    FLOOR(EXTRACT(EPOCH FROM c.event_dt) / 1209600) AS bin_14d,
    ROW_NUMBER() OVER (
      PARTITION BY d.ticker, FLOOR(EXTRACT(EPOCH FROM c.event_dt) / 1209600)
      ORDER BY c.event_dt DESC, c.transcriptid DESC
    ) AS rn
  FROM calls c
  JOIN dedup d USING (transcriptid)
)
SELECT
  transcriptid, event_dt, ticker, full_text
FROM bucketed
WHERE rn = 1
ORDER BY event_dt;
"""

tx_multi = db.raw_sql(sql_multi, params=params)
print(tx_multi.shape)
tx_multi.head(3)


(108, 4)


,transcriptid,event_dt,ticker,full_text
0,2866339.0,2023-08-02 20:39:31,TSLA,"Ladies and gentlemen, thank you for standing b..."
1,2866669.0,2023-08-02 23:26:13,GOOGL,"Good afternoon. My name is Sarah, and I will b..."
2,2886847.0,2023-08-16 17:31:29,TSLA,Welcome to today's earnings call of the Elevin...


In [13]:
tx = tx_multi.copy()

In [14]:

TICKER_COL = "ticker"
TEXT_COL   = "full_text"

# sanity checks
if not isinstance(tx_multi, pd.DataFrame):
    raise ValueError("tx_multi must be a pandas DataFrame.")
for col in [TICKER_COL, TEXT_COL, "event_dt", "transcriptid"]:
    if col not in tx_multi.columns:
        raise ValueError(f"'{col}' column missing in tx_multi.")

# derive tickers from the data (no aliasing)
tx_multi = tx_multi.copy()
tx_multi[TICKER_COL] = tx_multi[TICKER_COL].astype(str).str.upper()
TICKERS = sorted(tx_multi[TICKER_COL].dropna().unique().tolist())
print("TICKERS:", TICKERS)

# filter and clean
tx_filt = tx_multi[tx_multi[TICKER_COL].isin(TICKERS)].copy()
tx_filt["event_dt"] = pd.to_datetime(tx_filt["event_dt"], errors="coerce")
tx_filt = tx_filt.dropna(subset=["event_dt"])

# keep consistent columns
tx_filt = tx_filt[["transcriptid", "event_dt", TICKER_COL, TEXT_COL]].reset_index(drop=True)

print("tx_filt shape:", tx_filt.shape)
display(tx_filt.head())


TICKERS: ['GOOGL', 'TSLA']
tx_filt shape: (108, 4)


,transcriptid,event_dt,ticker,full_text
0,2866339.0,2023-08-02 20:39:31,TSLA,"Ladies and gentlemen, thank you for standing b..."
1,2866669.0,2023-08-02 23:26:13,GOOGL,"Good afternoon. My name is Sarah, and I will b..."
2,2886847.0,2023-08-16 17:31:29,TSLA,Welcome to today's earnings call of the Elevin...
3,2886964.0,2023-08-16 22:50:31,GOOGL,So post-lunch session. I'm Samik Chatterjee ag...
4,2899204.0,2023-08-30 14:30:26,TSLA,"Good day, everyone, and welcome to the full ye..."


In [15]:
# A) Global anchor + 14-day bins for transcripts
tx_filt = tx_filt.copy()
tx_filt["event_dt"] = pd.to_datetime(tx_filt["event_dt"], errors="coerce")
tx_filt = tx_filt.dropna(subset=["event_dt", "ticker"])

anchor = tx_filt["event_dt"].min()
tx_filt["interval"] = ((tx_filt["event_dt"] - anchor).dt.days // 14).astype(int)

# one transcript per (ticker, interval): most recent in that bin
by_bin_tx = (
    tx_filt.sort_values(["ticker", "event_dt"])
           .groupby(["ticker", "interval"], as_index=False)
           .tail(1)[["ticker","interval","event_dt","transcriptid","full_text"]]
           .sort_values(["ticker","interval"])
           .reset_index(drop=True)
)

print("Transcripts by-bin:", by_bin_tx.shape)
by_bin_tx.head()

Transcripts by-bin: (89, 5)


,ticker,interval,event_dt,transcriptid,full_text
0,GOOGL,0,2023-08-02 23:26:13,2866669.0,"Good afternoon. My name is Sarah, and I will b..."
1,GOOGL,1,2023-08-16 22:50:31,2886964.0,So post-lunch session. I'm Samik Chatterjee ag...
2,GOOGL,2,2023-08-30 22:54:08,2899513.0,"Hi. Good afternoon, everybody. We're going to ..."
3,GOOGL,3,2023-09-13 20:57:57,2907954.0,All right. I'd like to welcome everyone this a...
4,GOOGL,4,2023-09-27 21:49:02,2912862.0,"Good morning. Ladies and gentlemen, welcome to..."


In [16]:
# B) FINRA prep — align to the same anchor and 14-day grid
finra = finra_df.copy()  # <-- rename if needed
finra["settlementDate"] = pd.to_datetime(finra["settlementDate"], errors="coerce")
finra = finra.dropna(subset=["settlementDate"])

# harmonize ticker column name to match transcripts
finra = finra.rename(columns={"symbolCode": "ticker"})

# use the SAME anchor as transcripts
finra["interval"] = ((finra["settlementDate"] - anchor).dt.days // 14).astype(int)

# one FINRA row per (ticker, interval): latest settlement in that bin
by_bin_fi = (
    finra.sort_values(["ticker","settlementDate"])
         .groupby(["ticker","interval"], as_index=False)
         .tail(1)
)

print("FINRA by-bin:", by_bin_fi.shape)
by_bin_fi.head()

FINRA by-bin: (2197759, 15)


,accountingYearMonthNumber,ticker,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate,interval
2019402,20210115,A,Agilent Technologies Inc.,A,NYSE,2266746,2502573,NaN,1618485,1.40,NaN,-9.42,-235827,2021-01-15,-67
2000454,20210129,A,Agilent Technologies Inc.,A,NYSE,2272817,2266746,NaN,1446062,1.57,NaN,0.27,6071,2021-01-29,-66
1981098,20210212,A,Agilent Technologies Inc.,A,NYSE,2352823,2272817,NaN,1368962,1.72,NaN,3.52,80006,2021-02-12,-65
1961603,20210226,A,Agilent Technologies Inc.,A,NYSE,2380017,2352823,NaN,1816473,1.31,NaN,1.16,27194,2021-02-26,-64
1601381,20210315,A,Agilent Technologies Inc.,A,NYSE,2908967,2380017,NaN,1784590,1.63,NaN,22.22,528950,2021-03-15,-63


In [17]:
merged = (
    by_bin_tx.merge(by_bin_fi, on=["ticker","interval"], how="inner")
             .sort_values(["ticker","interval"])
             .reset_index(drop=True)
)

print("Merged shape:", merged.shape)
merged.head(3)

Merged shape: (81, 18)


,ticker,interval,event_dt,transcriptid,full_text,accountingYearMonthNumber,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate
0,GOOGL,0,2023-08-02 23:26:13,2866669.0,"Good afternoon. My name is Sarah, and I will b...",20230815,Alphabet Inc. Class A Common S,R,NNM,46172293,45223272,NaN,22857577,2.02,NaN,2.10,949021,2023-08-15
1,GOOGL,2,2023-08-30 22:54:08,2899513.0,"Hi. Good afternoon, everybody. We're going to ...",20230831,Alphabet Inc. Class A Common S,R,NNM,49039430,46172283,NaN,28430529,1.72,R,6.21,2867147,2023-08-31
2,GOOGL,3,2023-09-13 20:57:57,2907954.0,All right. I'd like to welcome everyone this a...,20230915,Alphabet Inc. Class A Common S,R,NNM,46080635,49039430,NaN,22512788,2.05,NaN,-6.03,-2958795,2023-09-15


In [18]:
# 0) (Re)install parquet support if needed
!pip install -q pyarrow

# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# 1) NLTK setup (run once per session)
import nltk
nltk.download('punkt')
# some newer NLTK installs also need this:
try:
    nltk.download('punkt_tab')
except:
    pass

from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np

# 2) Your function (use a conservative max_words to stay <512 tokens)
def chunk_text(text, max_words=220):
    if not isinstance(text, str) or not text.strip():
        return []
    sents = sent_tokenize(text)
    chunks, current, count = [], [], 0
    for sent in sents:
        wlen = len(word_tokenize(sent))
        if count + wlen > max_words:
            if current:
                chunks.append(" ".join(current))
            current = [sent]
            count = wlen
        else:
            current.append(sent)
            count += wlen
    if current:
        chunks.append(" ".join(current))
    return chunks

# 3) Build a document table and a chunk table
docs = merged.copy()                       # merged: one row per interval
docs["doc_id"] = np.arange(len(docs))
rows = []
for doc_id, txt in zip(docs["doc_id"], docs["full_text"].fillna("")):
    parts = chunk_text(txt, max_words=220)
    for j, ch in enumerate(parts):
        rows.append({
            "doc_id": doc_id,
            "chunk_id": j,
            "chunk_text": ch,
            "chunk_word_len": len(ch.split())
        })
chunks = pd.DataFrame(rows)

# 4) Save for scoring in NoLBERT notebook
OUT_DIR = "/content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs"
docs.to_parquet(f"{OUT_DIR}/merged_docs_new.parquet", index=False)
chunks.to_parquet(f"{OUT_DIR}/merged_chunks_nltk_new.parquet", index=False)
print("Saved:\n ", f"{OUT_DIR}/merged_docs.parquet", "\n ", f"{OUT_DIR}/merged_chunks_nltk_new.parquet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Saved:
  /content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs/merged_docs.parquet 
  /content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs/merged_chunks_nltk_new.parquet
